### PromptsTemplate in langchain <br> Templates with Placeholders

In [1]:
from langchain_core.prompts import PromptTemplate

tempalte="hey my name is {name} and I am {age} years old"
prompt=PromptTemplate.from_template(tempalte)

print("prompt type:", type(prompt))
print("inptut variables:", prompt.input_variables)
print(prompt.format(name="ujwal", age=22))

prompt type: <class 'langchain_core.prompts.prompt.PromptTemplate'>
inptut variables: ['age', 'name']
hey my name is ujwal and I am 22 years old


### Partial Prompts to define some Default var

In [2]:
tempalte="hey my name is {name} and I am {age} years old and I am a {profession}"
prompt=PromptTemplate.from_template(tempalte)

prompt_with_default=prompt.partial(name="ujwal", age=22)

#now provide only the profession
print(prompt_with_default.format(profession="developer"))

hey my name is ujwal and I am 22 years old and I am a developer


### Template with the multiple messages of CHAT models

In [3]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

chat_tempalte = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    HumanMessagePromptTemplate.from_template("{question}"),
])
chat=chat_tempalte.partial(question="What is LangChain?")
print(chat.format())

System: You are a helpful assistant
Human: What is LangChain?


### Few Shot prompting

In [4]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

exmaple=[
    {"input": "What is LangChain?", "output": "LangChain is a framework for building applications with LLMs."},
    {"input": "What is LangGraph?", "output": "LangGraph is a framework"}
]

example_prompt=ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessagePromptTemplate.from_template("{output}")
    ])
few_shot_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=exmaple,
)
final_prompt=ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    few_shot_prompt,
    HumanMessagePromptTemplate.from_template("{input}")
])
print(final_prompt.format(input="What is LangGraph?"))

System: You are a helpful assistant
Human: What is LangChain?
AI: LangChain is a framework for building applications with LLMs.
Human: What is LangGraph?
AI: LangGraph is a framework
Human: What is LangGraph?
